# Cours 11 : Scraping et APIs

Le web scraping est une technique d'extraction automatique de données depuis des sites web. En sciences humaines, on peut l'utiliser pour constituer des jeux de données (fichiers excel...) ou des corpus.

Dans ce cours, nous allons utiliser Python et quelque librairies qui nous permettront d'explorer des pages web, récupérer leur contenu et extraire les éléments qui nous interessent pour ensuite les sauvegarder dans des formats structurés exploitables.

Pour plus d'information au sujet du web scraping, vous pouvez consulter [cet article](https://realpython.com/beautiful-soup-web-scraper-python/)

## Structure d'une page web

Avant d'entamer le processus de scraping, il est important de comprendre la structure du site web.

Les pages web sont un ensemble de HTML (pour la structure), CSS (pour l'habillage) et finalement le JavaScript (pour l'interaction).

Voici un exemple de code HTML :

```
<html> <!-- Ceci est un commentaire , comme pour python mais sans le # -->
    
    <head> <!-- la balise style permet d'y ecrire du css -->
        <style>
            .ma-classe{
                color:red;
            }
        </style>

        <title>Ma-super-page.com</title>
    </head>
    <body>
        <h1 class="ma-classe">Mon titre principale</h1>
        <p>Mon paragraphe</p>
        <h2>Mon sous title</h2>
        ...

        <script>
// ici nous pourons ecrire du code javascript pour ajouter de l'animation ou de l'interaction
        </script>
    </body>
</html>
<!-- -->
```

Rassurez-vous, nous resterons sur du Python, mais il est important de voir comment se structure une page web afin de pouvoir la manipuler. Pour plus d'information à ce sujet, vous pouvez consulter [cette page](https://developer.mozilla.org/fr/docs/Learn_web_development/Core/Structuring_content).

Pour commencer, nous allons utiliser un outil présent sur toutes nos machines : notre navigateur et son inspecteur !

Prenons cette page : https://fr.wikipedia.org/wiki/UCLouvain et essayons d'en explorer la structure.

une fois sur la page utilisez le clic droit de votre souris et selectionnez l'inspecteur (_Inspect_ ou _Inspecter la page_).

À gauche, nous pouvons observer la structure HTML de la page, composée de différents blocs que nous désignons sous les termes de balises ou tags. À droite se trouvent les éléments CSS associés aux divers blocs HTML.

Pour faciliter l'exploration de chacun de ces blocs, notre inspecteur met à notre disposition plusieurs outils utiles. 

L'inspecteur permet aussi de modifier des éléments de la page (un titre, la couleur d'un paragraphe, une couleur de fond, une image...). On peut par exemple modifier le contenu d'une des balises en utilisant l'outil de sélection: changeons le titre de la page en "UCL"

## Les outils de scraping avec Python

Maintenant que nous avons exploré une page web manuellement, nous allons voir comment l'explorer avec du code Python afin d'automatiser ce travail. Pour ce faire, nous allons télécharger et installer deux bibliothèques indispensables pour le scraping de pages web : `requests` et `beautifulsoup`.

- `requests` nous permet d'effectuer des requêtes HTTP, et donc d'accéder aux pages web.- `beautifulsoup` nous aide à récupérer le contenu des différentes balises ou l'intégralité de la page dans des variables Python.

In [ ]:
!pip install bs4
!pip install requests

## Scrapons !

Dans les sections suivantes, vous trouverez les différentes étapes pour accéder à une page et récupérer le contenu de certaines balises. L'objectif ici est d'extraire le contenu que nous avons préalablement identifié en inspectant la page.

In [ ]:
from bs4 import BeautifulSoup
import requests

url ='https://fr.wikipedia.org/wiki/UCLouvain'
page = requests.get(url) # nous essayons juste d'acceder a la page web , comme si il s'agissait de notre navigateur

soup = BeautifulSoup(page.text,'html') # On charge le contenu de la page dans BeautifulSoup
print(soup.prettify()) # nous affichons l'entiereté de la page html (on utilise une fonction prettify pour rendre le tout un peu plus lisible)

Nous allons ensuite récupérer 2 informations: le titre de la page, ainsi que son contenu texte. Pour ce faire, nous allons utiliser les balises `h1` (titre) et `p` (paragraphe). Nous allons itérer à travers les paragraphes, et pour chacun de ceux-ci, imprimer son contenu.

In [ ]:
title = soup.find('h1') # je lui demande de retrouver la premiere balise h1
print(title.text) # je demande d'afficher le text contenu dans la balise si je ne le precise pas que va t'il ecrire selon vous ?

In [ ]:
for p in soup.find_all('p'):
    print(p.text)

## Exercice: récupération des articles de la page d'accueil de la RTBF

Pour cet exercice, nous allons créer un robot qui va ouvrir la page d'accueil du site de la[RTBF](https://www.rtbf.be/), récupérer le titre de tous les articles du jour et les stocker dans un fichier csv.

Commençons par importer les librairies nécessaires

In [ ]:
import os
import re
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd

### Récupération de tous les articles de la page d'accueil



In [ ]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

articles = []

root_url = f"https://www.rtbf.be"
response = requests.get(root_url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')
for link in soup.find_all("a", {"class":"stretched-link outline-none"}):
    title = link.text.strip()
    url = root_url + link.get('href')
    articles.append([url, title])

In [ ]:
# Affichage du nombre d'articles récupérés
len(articles)

In [ ]:
# Affichage des 10 premières entrées
articles[:10]

### Création d'un dataframe avec les liens et les titres des articles


In [ ]:
df = pd.DataFrame(articles, columns=['link', 'title'])
df

In [ ]:
# Sauvegarde du dataframe dans un fichier csv
df.to_csv(f"rtbf_{time.strftime('%Y%m%d')}.csv", index=False)

### Téléchargement d'un article et affichage du texte

In [ ]:
# Récupération du premier lien du dataframe
article_url = df.iloc[0][0]

# Ouverture de la page
response = requests.get(article_url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')
# Parsing du contenu
title = soup.find("h1").get_text(separator=' ')

paragraphs = []
for paragraph in soup.find_all("p", attrs={"class": None}):
    paragraphs.append(paragraph.get_text(separator=' ').strip())

content = "\n".join(paragraphs)
print(title)
print("==================================")
print(content)

### Nettoyage du texte à l'aide d'expressions régulières

In [ ]:
# Suppression de tout ce qui se trouve entre parenthèses
clean_content = re.sub("\([^\)]+\)", " ", content)
# Suppression des espaces multiples
clean_content = re.sub("\s+", " ", clean_content)

print(content)
print("==================================")
print(clean_content)


### Création d'un fichier avec le contenu de l'article


In [ ]:
with open("rtbf_example.txt", "w") as writer:
    writer.write(f"{title}\n\n{clean_content}")

## Utilisation d'APIs

## API (Application Programming Interface)

Une API (Interface de Programmation d’Application) constitue une alternative puissante au web scraping pour la création de jeux de données ou de corpus. Il s’agit d’un ensemble de méthodes qui permettent à des applications de communiquer entre elles, de manière structurée et fiable.

Alors qu’un humain interagit avec un logiciel via une interface utilisateur, une API permet à des programmes ou des services d’interagir directement entre eux. Par exemple, plutôt que d’extraire les informations visibles sur une page web (comme on le ferait avec du scraping), on peut interroger directement l’API d’un service pour obtenir des données propres, souvent mieux structurées et plus complètes.

De nombreuses entreprises mettent à disposition des API — gratuites ou payantes — qui permettent d’accéder à leurs données de manière automatisée. Il est souvent nécessaire de s’y authentifier, mais cela ouvre la porte à la création efficace de datasets sans avoir à manipuler le code HTML des pages web.


### Comment appeler une API ?

Chaque API a ses propres spécifications. En résumé, voici ce qu’il faut connaître :

- L’**URL racine**, qui définit l’adresse de l’API  
- Une **méthode** (les plus courantes sont `GET` et `POST`, mais il en existe beaucoup d'autres [ici](https://restfulapi.net/http-methods/))  
- Un **endpoint**, dont le rôle est comparable à une fonction en Python  
- Des **paramètres**, que l’on peut assimiler aux paramètres d’une fonction en Python  

En pratique, on peut tester une API directement dans le navigateur (au moins pour les méthodes `GET`). Un outil très utile pour tester des appels API plus complexes est [Postman](https://www.postman.com/).

Dans notre cas, nous utiliserons bien sûr Python et notre fameuse bibliothèque `requests`.


### Exercice

On va utiliser [newsapi](https://newsapi.org/) pour récupérer les derniers articles du journal français [L'Equipe](https://www.lequipe.fr/).

Dans notre cas les spécifications sont les suivantes :

- URL racine: `https://newsapi.org/v2`
- Méthode: `GET`
- Endpoint: `top-headlines`
- Paramètres (clé: valeur):
    - `apiKey`: `73bbb95f8ecb49b499113a46481b4af1` (this credential key has been created for you)
    - `sources`: `lequipe`

Dans le navigateur, voici la manière de traduire cette url : `{ROOT_URL}/{ENDPOINT}?{key}={value1}&{key2}={value2}`

Essayons ensemble de construire cette URL et de l'ouvrir dans le navigateur.

#### Et en Python ?

Pour faire ceci on peut directement utiliser la librairie `requests`

In [ ]:
import requests

root_url = ""
endpoint = "" 
params = {
    "apiKey": "",
    "sources": ""
}
# Let's call the get method of requests on our specifications
response = requests.get(f"{root_url}/{endpoint}", params=params)
response.json()

Le résultat est un peu indigeste, non ? Pourquoi ne pas analyser (parser) la sortie JSON pour obtenir uniquement les titres des articles dans une liste ?

In [ ]:
# Votre code ici